# Installing Dependencies

In [ ]:
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.7 MB/s eta 0:00:00


In [ ]:
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.7 MB/s eta 0:00:00


# Building Some Vectors

In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

## Loading the data

In [ ]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

data = pd.read_csv(StringIO(res.text), sep='\t')
data.head

<bound method NDFrame.head of       pair_ID                                         sentence_A  \
0           1  A group of kids is playing in a yard and an ol...   
1           2  A group of children is playing in the house an...   
2           3  The young boys are playing outdoors and the ma...   
3           5  The kids are playing outdoors near a man with ...   
4           9  The young boys are playing outdoors and the ma...   
...       ...                                                ...   
4495     9993                    A door is being opened by a man   
4496     9997                   Someone is boiling okra in a pot   
4497     9998  The man is singing heartily and playing the gu...   
4498     9999        A man in blue has a yellow ball in the mitt   
4499    10000               Three dogs are resting on a sidewalk   

                                             sentence_B  relatedness_score  \
0     A group of boys in a yard is playing and a man...                4.5 

In [ ]:
# take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
sentences_b = data['sentence_B'].tolist()
sentences.extend(sentences_b)

In [ ]:
len(set(sentences))

4802

## Add more data's

This is not particularly large number, so pull more data from websites

In [ ]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [ ]:
# add all the data

for url in urls:
  res = requests.get(url)
  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
  sentences.extend(data[1].tolist())
  sentences.extend(data[2].tolist())

<ipython-input-10-cd23f4b32abd>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-10-cd23f4b32abd>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-10-cd23f4b32abd>:5: F

In [ ]:
len(set(sentences))

14505

## Removing duplicates

Build Dense vector representations fo each sentences using sentence-BERT

In [ ]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
sentences[0]

'The men are playing soccer'

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

# Plain and Simple

### IndeFlatL2

It measures the L2 or Euclidean distance between all given points between our query vector, and the vector loaded into the index, It's simple, very accurate, but not too fast.

In [ ]:
import faiss

In [ ]:
d = sentence_embeddings.shape[1]
d

768

In [ ]:
index = faiss.IndexFlatL2(d)

Before loading in our data. We can whether an index needs to be trained using `is_trained` method.
`IndexFlatL2` is not index that requires training, so it should return False

In [ ]:
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

14504

The given query `xq` and number of nearest neighbors to return `k`

In [ ]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 7703    43 10073  3386]]
CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 17.4 ms


In [ ]:
out = pd.DataFrame(sentences, columns=['sentences'])

In [ ]:
out.iloc[[7703,43,10073,3386]]

,sentences
7703,A group of football players is running in the ...
43,A group of people playing football is running ...
10073,Two groups of people are playing football
3386,A person playing football is running past an o...


In [ ]:
vecs = np.zeros((k,d))
for i, val in enumerate(I[0].tolist()):
  vecs[i, :] = index.reconstruct(val)

In [ ]:
vecs.shape

(4, 768)

In [ ]:
vecs[0][:100]

array([ 0.01627043,  0.22325923, -0.15037429, -0.30747262, -0.27122447,
       -0.10593179, -0.06460944,  0.04738191, -0.73349023, -0.37657711,
       -0.76762801,  0.16902894,  0.53107643,  0.51176697,  1.14415872,
       -0.08562925, -0.67240071, -0.966371  ,  0.0254543 , -0.21559827,
       -1.25656569, -0.82982153, -0.09825017, -0.21850844,  0.50610238,
        0.10527948,  0.50396854,  0.6524291 , -1.39458752,  0.65847468,
       -0.21525319, -0.22487448,  0.81818295,  0.08464289, -0.76141745,
       -0.28928283, -0.0982582 , -0.73046213,  0.07855788, -0.84354621,
       -0.59242105,  0.77471346, -1.20920527, -0.22757953, -1.30733597,
       -0.2308148 , -1.31322539,  0.01629077, -0.97285479,  0.19308184,
        0.47424543,  1.18920851, -1.96741307, -0.70061129, -0.29638696,
        0.60533702,  0.62407428, -0.70340371, -0.86754227,  0.17673181,
       -0.19170485, -0.02952001,  0.2262357 , -0.16695452, -0.80402535,
       -0.45918915,  0.69675493, -0.24928176, -1.0147866 , -0.92

# Speed

`IndexFlatL2` index alone is computationally expensive, it doesn't scale well.

when using index we perform exhaustive search - comparing the `xq` with all the parameters of out index.

# Partitioning The Index

Using this method, we would take a query vector `xq` identify the cell it belong to and then use our `IndexFlatL2` to search between the query vector and all other vectors belonging to that specific cell.

Reducing the scope of search to produce approximate answer rather than accurate, it is less exhaustive search

**Implementation**

L2 index (quantizer step) --> `IndexIVFFlat` index

`nlist` - No.of partitions

`IndexFlatL2` - index, need training and grouping/transformations to build index
Because we added clustering with `IndexIVFFlat` this is no longer class

In [ ]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search the same indexed sentence embeddings and the same query vector `xq`

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 7703    43  3117 12029]]
CPU times: user 1.13 ms, sys: 895 µs, total: 2.02 ms
Wall time: 3.17 ms


In [ ]:
out.iloc[[7703,43,3117,12029]]

,sentences
7703,A group of football players is running in the ...
43,A group of people playing football is running ...
3117,A group of football players running down the f...
12029,A man is wearing a sports uniform and is runni...


If approximate search with `IndexIVFFlat` return suboptimal results, we can improve accuracy by increasing search scope.
we do this by increasing the `nprobe` attribute value

In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 7703    43 10073  3386]]
CPU times: user 5.17 ms, sys: 276 µs, total: 5.45 ms
Wall time: 8.8 ms


In [ ]:
out.iloc[[7703, 43, 10073, 3386]]

,sentences
7703,A group of football players is running in the ...
43,A group of people playing football is running ...
10073,Two groups of people are playing football
3386,A person playing football is running past an o...


# Vector Reconstruction

If we use `index.reconstruct(<vector_idx)` again, we will return a `RuntimeError`
because there is no mapping between original vectors and their index positions, due to addition of the IVF step.

Need to recontruct the vectors, we must create these direct mapping using `index.make_direct_map()`

In [ ]:
index.make_direct_map()

In [ ]:
index.reconstruct(7460)[:100]

array([-0.03574951,  0.12185385,  0.96285826,  0.0488941 ,  0.07329725,
       -1.089388  , -0.297118  ,  0.4359192 ,  0.55184007, -0.1835841 ,
       -0.24265243, -0.0411182 ,  0.9429575 ,  0.20833752,  0.18138924,
        0.21035567, -0.07310223, -0.23709436,  0.1395393 , -0.33101612,
       -0.08951158, -1.5015551 , -0.08666448,  0.7150375 ,  1.0592242 ,
        0.44194388, -0.10452043, -0.4104125 , -0.61108303, -0.19135678,
       -0.18667264, -0.07612804, -1.0352876 ,  0.01216166, -0.44802403,
        1.0959883 , -0.2770376 ,  0.43440828, -0.66375774, -0.180599  ,
       -0.17489687, -0.47711882,  0.57493824,  0.05606867, -1.080574  ,
       -0.92719835,  0.7883081 ,  0.10364299, -0.51560545, -0.7037303 ,
        0.21890233,  0.13906085, -0.23535666, -0.27976808, -0.6274319 ,
        0.20470661, -0.37923545, -1.1218337 , -1.1337851 , -0.1107201 ,
       -0.95790166, -0.19208574,  0.19626585,  0.39056122, -0.22048537,
       -0.02556027,  0.07827501,  0.38407528, -0.3750913 , -1.27

# Quantization

Faiss comes with compress ability of our vector using product quanitzation (PQ)


PQ achieves this approximated similarity operation by compressing the vectors themselves, which consists of three steps.

**Original vector** ---> **1. Sliced sub-vectors** ---> **2. slice clustering** ---> **3. Centroid ID vector**

1. split original vector into several subvectors.
2. for each sub vector, perform cluster operation - creating mutliple centroid for each sub-vector set.
3. Replace the sub-vector with the ID of it's nearest set-specific centroid,

In [ ]:
m = 8
bits = 8

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.nprobe = 10

In [ ]:
%%time
D,I = index.search(xq, k)
print(I)

[[ 346 2173 2784 1840]]
CPU times: user 2.07 ms, sys: 121 µs, total: 2.19 ms
Wall time: 6.02 ms


In [ ]:
out.iloc[[346, 2173, 2784, 1840]]

,sentences
346,Two football players are scrambling for the ba...
2173,some football players in red jerseys taking an...
2784,A football player is carrying an official past...
1840,Two teams are competing in a football match


# Speed or Accuracy

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['346: Two football players are scrambling for the ball on the court',
 '2173: some football players in red jerseys taking another in a white jersey to the ground',
 '2784: A football player is carrying an official past a rolling football',
 '1840: Two teams are competing in a football match']